## Preprocessing

In [ ]:
conda activate qiime2-2021.4

In [2]:
mkdir 2021_BNW_skin
mkdir 16SV4_all
cd 16SV4_all

In [ ]:
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \ 
  --input-path /Users/christinanaesborg/Desktop/Raw_Samples/16SV4_BNW/Collected \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux.qza

In [ ]:
qiime demux summarize \
  --i-data demux-paired-end.qza \
  --o-visualization demux-paired-end.qzv

qiime tools view demux-paired-end.qzv

In [ ]:
qiime metadata tabulate \
--m-input-file BNW-skin-stats-dada2.qza \
--o-visualization BNW-skin-stats-dada2.qzv 
qiime feature-table summarize \
--i-table BNW-skin-table.qza \
--o-visualization BNW-skin-table.qzv
qiime feature-table tabulate-seqs \
--i-data BNW-skin-rep-seqs.qza \
--o-visualization BNW-skin-rep-seqs.qzv

In [ ]:
# since sample id were only numbers, I had to reindex. Otherwise I would have incountered problems later on
qiime feature-table group \
--i-table BNW-skin-table.qza \
--p-axis sample \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column SampleID-new \
--p-mode sum \
--o-grouped-table BNW-skin-reindexed-table.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-reindexed-table.qza \
--m-sample-metadata-file BNW-skin-metadata.tsv \
--o-visualization BNW-skin-reindexed-table.qzv

# Training SILVA database

In [ ]:
# import Qiime2 artifacts
qiime tools import \
  --type 'FeatureData[Sequence]' \
  --input-path /Users/christinanaesborg/Desktop/SILVA_132_QIIME_release/rep_set/rep_set_16S_only/99/silva_132_99_16S.fna  \
  --output-path silva_99_otus.qza

In [ ]:
qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --input-format HeaderlessTSVTaxonomyFormat \
  --input-path /Users/christinanaesborg/Desktop/SILVA_132_QIIME_release/taxonomy/16S_only/99/taxonomy_all_levels.txt  \        
  --output-path silva_99_ref-taxonomy.qza

In [ ]:
# extract reference reads
qiime feature-classifier extract-reads \
  --i-sequences silva_99_otus.qza \
  --p-f-primer GTGCCAGCMGCCGCGGTAA \
  --p-r-primer GGACTACHVGGGTWTCTAAT \
  --p-trunc-len 253  \
  --p-min-length 100 \
  --p-max-length 400 \
  --o-reads silva_ref-seqs.qza

In [ ]:
# train naive bayes classifier
qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads silva_ref-seqs.qza \
  --i-reference-taxonomy silva_99_ref-taxonomy.qza \
  --o-classifier silva_132_99_classifier.qza

In [ ]:
qiime feature-classifier classify-sklearn \
  --i-classifier silva_132_99_classifier.qza \
  --i-reads BNW-skin-rep-seqs.qza \
  --o-classification BNW-skin-silva-132.qza

In [ ]:
qiime metadata tabulate \
--m-input-file BNW-skin-silva-132.qza \
--o-visualization BNW-skin-silva-132.qzv

# Pre-trained SILVA database

In [ ]:
qiime feature-classifier classify-sklearn \
--i-classifier silva-138-99-515-806-nb-classifier.qza \
--i-reads BNW-skin-rep-seqs.qza \
--o-classification BNW-skin-silva-138-taxonomy.qza

In [ ]:
qiime metadata tabulate \
--m-input-file BNW-skin-silva-138-taxonomy.qza \
--o-visualization BNW-skin-silva-138-taxonomy.qzv

In [ ]:
# Create phylogenetic tree using sepp. Took a couple of days to run. 
#Output files: BNW-skin-sepp-tree.qza BNW-skin-sepp-placements.qza
qiime fragment-insertion sepp \
--i-representative-sequences BNW-skin-rep-seqs.qza \
--i-reference-database sepp-refs-silva-128.qza \
--o-tree BNW-skin-sepp-tree.qza \
--o-placements BNW-skin-sepp-placements.qza \
--verbose \
--p-threads 2

In [ ]:
qiime fragment-insertion filter-features \
--i-table BNW-skin-table.qza \
--i-tree BNW-skin-sepp-tree.qza \
--o-filtered-table BNW-skin-filtered_table.qza \
--o-removed-table BNW-skin-removed_table.qza \
--verbose

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-removed_table.qza \
--o-visualization BNW-skin-removed_table.qzv

In [ ]:
Looks like nothing was removed, so will just use the unfiltered table (BNW-skin-table.qza) for subsequent analyses

In [ ]:
qiime diversity alpha-rarefaction \
--i-table BNW-skin-reindexed-table.qza \
--m-metadata-file /Users/christinanaesborg/Desktop/Microbiome/BNW-skin-metadata.tsv \
--o-visualization BNW-skin-reindexed-table-rarefaction.qzv \
--p-min-depth 500 \
--p-max-depth 50000

# output: BNW-skin-reindexed-table-rarefaction.qzv

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-reindexed-table.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-min-frequency 12201 \
--o-filtered-table BNW-table-atleast-12201-reads.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-table-atleast-12201-reads.qza \
--o-visualization BNW-table-atleast-12201-reads.qzv

# Remove toe samples and groups of Uncertain/low mange and Observational healthy from analysis as well as controls

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-reindexed-table.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--p-where "[bodysite] IN ('toe') OR [Severity] IN ('Uncertain/low mange', 'Observational healthy')" \
--p-exclude-ids \
--o-filtered-table BNW-skin-table_Final-sev.qza

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-table_Final-sev.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--p-where "[sampletype]='Control'" \
--p-exclude-ids \
--o-filtered-table BNW-skin-table_Final.qza

In [ ]:
# remove samples with low reads
qiime feature-table filter-samples \
--i-table BNW-skin-table_Final.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--p-min-frequency 12201 \
--o-filtered-table BNW-skin-table-at-least-12201-reads_Final.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-at-least-12201-reads_Final.qza \
--o-visualization BNW-skin-table-at-least-12201-reads_Final.qzv 

In [ ]:
# create taxa barplot per individual sample
qiime taxa barplot \
--i-table BNW-skin-table-at-least-12201-reads_Final.qza \
--i-taxonomy BNW-skin-silva-138-taxonomy.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--o-visualization BNW-skin-table-at-least-12201-reads-Final_BARPLOT-PER-SAMPLE.qzv

In [ ]:
# collapse samples by severity and make barplots
qiime feature-table group \
--i-table BNW-skin-table-at-least-12201-reads_Final.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--m-metadata-column Severity \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-skin-table-SEVERITY_Final.qza

In [ ]:
qiime taxa barplot \
--i-table BNW-skin-table-SEVERITY_Final.qza \
--i-taxonomy BNW-skin-silva-138-taxonomy.qza \
--m-metadata-file BNW-skin-metadata-SEVERITY.tsv \
--o-visualization BNW-skin-table-Final-SEVERITY-BARPLOT.qzv

In [ ]:
# core diversity on filtered table using 13084 rarefaction depth
qiime diversity core-metrics-phylogenetic \
--i-table BNW-skin-table-at-least-12201-reads_Final.qza \
--i-phylogeny BNW-skin-sepp-tree.qza \
--p-sampling-depth 13084 \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--output-dir BNW-skin-Core-metrics-final-filtered-table-13084

In [ ]:
for i in BNW-skin-Core-metrics-final-filtered-table-13084/*vector.qza; do qiime diversity alpha-group-significance --i-alpha-diversity $i --m-metadata-file BNW-skin-metadata.tsv --o-visualization ${i/.qza/.qzv}; done

In [ ]:
qiime diversity alpha \
  --i-table BNW-skin-table-at-least-12201-reads_Final.qza \
  --p-metric chao1 \
  --o-alpha-diversity BNW-skin-Core-metrics-final-filtered-table-13084/chao1_index.qza

In [ ]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity BNW-skin-Core-metrics-final-filtered-table-13084/chao1_index.qza \
  --m-metadata-file BNW-skin-metadata-16S.tsv \
  --o-visualization BNW-skin-Core-metrics-final-filtered-table-13084/chao1_index.qzv

# Permanova test for microbial composition differences

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix BNW-skin-Core-metrics-final-filtered-table-13084/bray_curtis_distance_matrix.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--m-metadata-column Severity \
--o-visualization BNW-skin-Core-metrics-final-filtered-table-13084/bray_curtis_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix BNW-skin-Core-metrics-final-filtered-table-13084/jaccard_distance_matrix.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--m-metadata-column Severity \
--o-visualization BNW-skin-Core-metrics-final-filtered-table-13084/jaccard_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

# ANCOM analysis to identify differentially abundant features/OTUS

In [ ]:
#remove low abundance/prevalence features
qiime feature-table filter-features \
--i-table BNW-skin-table-at-least-12201-reads_Final.qza \
--o-filtered-table BNW-skin-table-FILTERED-ANCOM-ms8-mf500_Final.qza \
--p-min-samples 8 \
--p-min-frequency 500

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-FILTERED-ANCOM-ms8-mf500_Final.qza \
--o-visualization BNW-skin-table-FILTERED-ANCOM-ms8-mf500_Final.qzv

In [ ]:
qiime composition add-pseudocount \
--i-table BNW-skin-table-FILTERED-ANCOM-ms8-mf500_Final.qza \
--o-composition-table BNW-skin-table-FILTERED-ANCOM-ms8-mf500-pseudo_Final.qza

In [ ]:
qiime composition ancom \
--i-table BNW-skin-table-FILTERED-ANCOM-ms8-mf500-pseudo_Final.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--m-metadata-column Severity \
--o-visualization BNW-skin-table-FILTERED-ANCOM-ms8-mf500-pseudo-SEVERITY_Final.qzv

# ANCOM collapsed at family level (i.e. not at the feature level)

In [ ]:
qiime taxa collapse \
--i-table BNW-skin-table-at-least-12201-reads_Final.qza  \
--i-taxonomy BNW-skin-silva-138-taxonomy.qza \
--p-level 5 \
--o-collapsed-table BNW-skin-table-FAMILY_Final.qza

In [ ]:
qiime feature-table filter-features \
--i-table BNW-skin-table-FAMILY_Final.qza \
--o-filtered-table BNW-skin-table-FAMILY-FILTERED-ANCOM-ms8-mf500_Final.qza \
--p-min-samples 8 \
--p-min-frequency 500

In [ ]:
qiime composition add-pseudocount \
--i-table BNW-skin-table-FAMILY-FILTERED-ANCOM-ms8-mf500_Final.qza  \
--o-composition-table BNW-skin-table-FAMILY-FILTERED-ANCOM-ms8-mf500-pseudo_Final.qza 

In [ ]:
qiime composition ancom \
--i-table BNW-skin-table-FAMILY-FILTERED-ANCOM-ms8-mf500-pseudo_Final.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--m-metadata-column Severity \
--o-visualization BNW-skin-table-FAMILY-FILTERED-ANCOM-ms8-mf500-pseudo-SEVERITY_Final.qzv

# Unique feature analysis

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-table-at-least-12201-reads_Final.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--p-where "[Severity]='Confidently healthy'" \
--o-filtered-table BNW-skin-table-CONFHEALTHYonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-CONFHEALTHYonly.qza \
--o-visualization BNW-skin-table-CONFHEALTHYonly.qzv

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-table-at-least-12201-reads_Final.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--p-where "[Severity]='Mangy'" \
--o-filtered-table BNW-skin-table-MANGYonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-MANGYonly.qza \
--o-visualization BNW-skin-table-MANGYonly.qzv

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-table-at-least-12201-reads_Final.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--p-where "[Severity]='Severe mange'" \
--o-filtered-table BNW-skin-table-SEVEREonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-SEVEREonly.qza \
--o-visualization BNW-skin-table-SEVEREonly.qzv

# Mitigating cross contamination/filter out common contaminats

# Pull out the NTCs to check if these abundant features spilled over into them as would be expected if cross-sample contamination was happening.

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-reindexed-table.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--p-where "SampleOrControl IN ('Negative control')" \
--o-filtered-table BNW-skin-table-NTCs-ONLY.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-NTCs-ONLY.qza \
--o-visualization BNW-skin-table-NTCs-ONLY.qzv

In [ ]:
qiime tools export \
--input-path BNW-skin-table-NTCs-ONLY.qza \
--output-path BNW-skin-table-NTCs-ONLY.biom

In [ ]:
biom convert -i BNW-skin-table-NTCs-ONLY.biom/feature-table.biom -o BNW-skin-table-NTCs-ONLY.biom/feature-table.tsv --to-ts

In [ ]:
biom convert -i bray_curtis.biom/feature-table.biom -o bray_curtis.biom/feature-table.tsv --to-tsv

In [ ]:
# There seems to be some cross-contamination. 
# Will compare BNW-skin-reindexed-table.qza to BNW-skin-table-NTCs-ONLY.qza
# to identify low abundance microbes as these are most likely contaminants arrising from extraction or sequencing.
# High abundance microbes could be a transfer from true biological samples, and will be retained in further analysis. 

In [ ]:
qiime taxa filter-table \
  --i-table BNW-skin-table_Final.qza \
  --i-taxonomy BNW-skin-silva-138-taxonomy.qza \
  --p-exclude mitochondria,8e09caaee61bd482522f087e96e093de,30ad487e1e86be71afce4e03fd7553b5,107273e73071282e4d0f4a0b46548da5,516a67156dacfa4341327a7e2a22f877,f2e69758d4f38addecad9194d39796e1,c489a5dd3e6b99cbbe986a7965f907ff,9ed691d9294238d730b23ab5b461835b,d829bee4984f82ffc2453212157caf96,882b5b49ef18215727a0900958cdec5d,f8485916bce858f455e79011d9a001f2,26ba1c3be9fb102889afcb9d61e730e7,06a7fe5af160203d234638f31f792cd0 \
  --o-filtered-table table-no-contaminants.qza

In [ ]:
qiime metadata tabulate \
--m-input-file BNW-skin-silva-138-taxonomy.qza \
--o-visualization BNW-skin-silva-138-taxonomy.qzv

In [ ]:
# remove samples with low reads
qiime feature-table filter-samples \
--i-table table-no-contaminants.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--p-min-frequency 12201 \
--o-filtered-table BNW-skin-table-no-contaminants-at-least-12201-reads_Final.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-no-contaminants-at-least-12201-reads_Final.qza \
--o-visualization BNW-skin-table-no-contaminants-at-least-12201-reads_Final.qzv 

In [ ]:
# convert to biom file for further analysis in R
qiime tools export \
--input-path BNW-skin-table-no-contaminants-at-least-12201-reads_Final.qza \
--output-path BNW-skin-table-no-contaminants-at-least-12201-reads_Final.biom

In [ ]:
# create taxa barplot per individual sample
qiime taxa barplot \
--i-table BNW-skin-table-no-contaminants-at-least-12201-reads_Final.qza \
--i-taxonomy BNW-skin-silva-138-taxonomy.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--o-visualization BNW-skin-table-no-cont-at-least-12201-reads-Final_BARPLOT-PER-SAMPLE.qzv

In [ ]:
# collapse samples by severity and make barplots
qiime feature-table group \
--i-table BNW-skin-table-no-contaminants-at-least-12201-reads_Final.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--m-metadata-column Severity \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-skin-table-no-cont-SEVERITY_Final.qza

In [ ]:
qiime taxa barplot \
--i-table BNW-skin-table-no-cont-SEVERITY_Final.qza \
--i-taxonomy BNW-skin-silva-138-taxonomy.qza \
--m-metadata-file BNW-skin-metadata-SEVERITY.tsv \
--o-visualization BNW-skin-table-Final-no-cont-SEVERITY-BARPLOT.qzv

In [ ]:
# core diversity on filtered table using 13084 rarefaction depth
qiime diversity core-metrics-phylogenetic \
--i-table BNW-skin-table-no-contaminants-at-least-12201-reads_Final.qza \
--i-phylogeny BNW-skin-sepp-tree.qza \
--p-sampling-depth 13084 \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--output-dir BNW-skin-Core-metrics-NoCont-filtered-table-13084

In [ ]:
for i in BNW-skin-Core-metrics-NoCont-filtered-table-13084/*vector.qza; do qiime diversity alpha-group-significance --i-alpha-diversity $i --m-metadata-file BNW-skin-metadata-16S.tsv --o-visualization ${i/.qza/.qzv}; done

In [ ]:
qiime diversity alpha \
  --i-table BNW-skin-table-no-contaminants-at-least-12201-reads_Final.qza \
  --p-metric chao1 \
  --o-alpha-diversity BNW-skin-Core-metrics-NoCont-filtered-table-13084/chao1_index.qza

In [ ]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity BNW-skin-Core-metrics-NoCont-filtered-table-13084/chao1_index.qza \
  --m-metadata-file BNW-skin-metadata-16S.tsv \
  --o-visualization BNW-skin-Core-metrics-NoCont-filtered-table-13084/chao1_index.qzv

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix BNW-skin-Core-metrics-NoCont-filtered-table-13084/bray_curtis_distance_matrix.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--m-metadata-column Severity \
--o-visualization BNW-skin-Core-metrics-NoCont-filtered-table-13084/bray_curtis_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix BNW-skin-Core-metrics-NoCont-filtered-table-13084/jaccard_distance_matrix.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--m-metadata-column Severity \
--o-visualization BNW-skin-Core-metrics-NoCont-filtered-table-13084/jaccard_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

# Unique feature analysis of severity groupings without contaminants - split into the three severity groupings

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-table-no-contaminants-at-least-12201-reads_Final.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--p-where "[Severity]='Confidently healthy'" \
--o-filtered-table BNW-skin-table-NoCont-CONFHEALTHYonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-NoCont-CONFHEALTHYonly.qza \
--o-visualization BNW-skin-table-NoCont-CONFHEALTHYonly.qzv

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-table-no-contaminants-at-least-12201-reads_Final.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--p-where "[Severity]='Mangy'" \
--o-filtered-table BNW-skin-table-NoCont-MANGYonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-NoCont-MANGYonly.qza \
--o-visualization BNW-skin-table-NoCont-MANGYonly.qzv

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-table-no-contaminants-at-least-12201-reads_Final.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--p-where "[Severity]='Severe mange'" \
--o-filtered-table BNW-skin-table-NoCont-SEVEREonly.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-NoCont-SEVEREonly.qza \
--o-visualization BNW-skin-table-NoCont-SEVEREonly.qzv

# Provide overview of ASVs for each severity type

In [ ]:
qiime tools export \
--input-path BNW-skin-table-NoCont-CONFHEALTHYonly.qza \
--output-path BNW-skin-table-NoCont-CONFHEALTHYonly.biom

In [ ]:
biom convert -i BNW-skin-table-NoCont-CONFHEALTHYonly.biom/feature-table.biom -o BNW-skin-table-NoCont-CONFHEALTHYonly.biom/feature-table.tsv --to-tsv

In [ ]:
qiime tools export \
--input-path BNW-skin-table-NoCont-MANGYonly.qza \
--output-path BNW-skin-table-NoCont-MANGYonly.biom

In [ ]:
biom convert -i BNW-skin-table-NoCont-MANGYonly.biom/feature-table.biom -o BNW-skin-table-NoCont-MANGYonly.biom/feature-table.tsv --to-tsv

In [ ]:
qiime tools export \
--input-path BNW-skin-table-NoCont-SEVEREonly.qza \
--output-path BNW-skin-table-NoCont-SEVEREonly.biom

In [ ]:
biom convert -i BNW-skin-table-NoCont-SEVEREonly.biom/feature-table.biom -o BNW-skin-table-NoCont-SEVEREonly.biom/feature-table.tsv --to-tsv

# Unique ASVs - min 3 samples due to severity groupings

In [ ]:
qiime feature-table filter-features \
--i-table BNW-skin-table-NoCont-CONFHEALTHYonly.qza \
--o-filtered-table BNW-skin-table-NoCont-CONFHEALTHYonly-ms3.qza \
--p-min-samples 3

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-NoCont-CONFHEALTHYonly-ms3.qza \
--o-visualization BNW-skin-table-NoCont-CONFHEALTHYonly-ms3.qzv

# Pre-filter
Number of features: 2807
Total frequency: 950,162

# Post-filter
Number of features: 280
Total frequency: 738,131

In [ ]:
qiime feature-table filter-features \
--i-table BNW-skin-table-NoCont-MANGYonly.qza \
--o-filtered-table BNW-skin-table-NoCont-MANGYonly-ms3.qza \
--p-min-samples 3

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-NoCont-MANGYonly-ms3.qza \
--o-visualization BNW-skin-table-NoCont-MANGYonly-ms3.qzv

# Pre-filter 
Number of features: 870 Total frequency: 374,474
# Post-filter 
Number of features: 34 Total frequency: 151,855

In [ ]:
qiime feature-table filter-features \
--i-table BNW-skin-table-NoCont-SEVEREonly.qza \
--o-filtered-table BNW-skin-table-NoCont-SEVEREonly-ms3.qza \
--p-min-samples 3

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-NoCont-SEVEREonly-ms3.qza \
--o-visualization BNW-skin-table-NoCont-SEVEREonly-ms3.qzv

# Prefilter 
Number of features: 496 Total frequency: 736,726
# Post-filter 
Number of features: 96 Total frequency: 583,522

# Merge groups for taxa barplots

In [ ]:
qiime feature-table group \
--i-table BNW-skin-table-NoCont-CONFHEALTHYonly-ms3.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--m-metadata-column wombatid \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-skin-table-NoCont-CONFHEALTHY-ms3-wombat-COLLAPSED.qza

In [ ]:
qiime feature-table group \
--i-table BNW-skin-table-NoCont-MANGYonly-ms3.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--m-metadata-column wombatid \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-skin-table-NoCont-MANGY-ms3-wombat-COLLAPSED.qza

In [ ]:
qiime feature-table group \
--i-table BNW-skin-table-NoCont-SEVEREonly-ms3.qza \
--m-metadata-file BNW-skin-metadata-16S.tsv \
--m-metadata-column wombatid \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-skin-table-NoCont-SEVERE-ms3-wombat-COLLAPSED.qza

In [ ]:
qiime feature-table merge \
--i-tables BNW-skin-table-NoCont-CONFHEALTHY-ms3-wombat-COLLAPSED.qza BNW-skin-table-NoCont-MANGY-ms3-wombat-COLLAPSED.qza BNW-skin-table-NoCont-SEVERE-ms3-wombat-COLLAPSED.qza \
--o-merged-table BNW-skin-table-NoCont-wombat-COLLASPED.qza